# OPTAA Calibration Script
This script provides an example of using the OPTAA Calibration parser for writing a calibration csv in the appropriate manner for OOInet to be pushed to asset management.

In [1]:
from utils import *

In [265]:
import shutil
import string

In [38]:
def get_calibration_files(serial_nums,dirpath):
    """
    Function which gets all the calibration files associated with the
    instrument serial numbers.
    
    Args:
        serial_nums - serial numbers of the instruments
        dirpath - path to the directory containing the calibration files
    Returns:
        calibration_files - a dictionary of instrument uids with associated
            calibration files
    """
    calibration_files = {}
    for uid in serial_nums.keys():
        sn = serial_nums.get(uid)[0]
        print(sn)
        files = []
        for file in os.listdir(dirpath):
            if sn in file:
                if 'Calibration_File' in file:
                    files.append(file)
                else:
                    pass
            else:
                pass
        
        calibration_files.update({uid:files})
        
    return calibration_files

In [250]:
def ensure_dir(file_path):
    """
    Function which checks that the directory where you want
    to save a file exists. If it doesn't, it creates the 
    directory.
    """
    if not os.path.exists(file_path):
        os.makedirs(file_path)

In [261]:
class OPTAACalibration():
    
    def __init__(self, uid):
        self.serial = ''
        self.uid = uid
        self.date = None
        self.cwlngth = []
        self.awlngth = []
        self.tcal = None
        self.tbins = None
        self.ccwo = []
        self.acwo = []
        self.tcarray = []
        self.taarray = []
        self.nbins = None  # number of temperature bins
        self.coefficients = {'CC_taarray': 'SheetRef:CC_taarray',
                             'CC_tcarray': 'SheetRef:CC_tcarray'}
        
    @property
    def uid(self):
        return self._uid
        
    @uid.setter
    def uid(self, d):
        r = re.compile('.{5}-.{6}-.{5}')
        if r.match(d) is not None:
            self.serial = 'ACS-' + d.split('-')[2].strip('0')
            self._uid = d       
        else:
            raise Exception(f"The instrument uid {d} is not a valid uid. Please check.")
            

    def load_dev(self, filepath):
        """
        Function loads the dev file for the OPTAA.
        
        Args:
            filepath - the full path, including the name of the file, to the optaa
                dev file.
        Returns:
            self.date - the date of calibration
            self.tcal - calibration temperature
            self.nbins - number of temperature bins
            self.cwlngth
            self.awlngth
            self.ccwo
            self.acwo
            self.tcarray
            self.taarray
            self.coefficients - a dictionary of the calibration values and associated
                keys following the OOI csv naming convention

        """
        
        
        if filepath.endswith('.zip'):
            with ZipFile(filepath) as zfile:
                filename = [name for name in zfile.namelist() if name.endswith('.dev')]
                text = zfile.read(filename[0]).decode('ASCII')
        
        else:
            with open(filepath) as file:
                text = file.read()
            
        # Remove extraneous characters from the 
        punctuation = ''.join((letter for letter in string.punctuation if letter not in ';/.'))
        
        for line in text.replace('\t',' ').splitlines():
            line = ''.join((word for word in line if word not in punctuation))
            
            if 'tcal' in line:
                data = line.split()
                # Temperature calibration value
                tcal = data.index('tcal')
                self.tcal = data[tcal+1]
                self.coefficients['CC_tcal'] = self.tcal
                # Temperature calibration date
                cal_date = data[-1].strip()
                self.date = pd.to_datetime(cal_date).strftime('%Y%m%d')
                
            elif ';' in line:
                data, comment = line.split(';')
        
                if 'temperature bins' in comment:
                    if 'number' in comment:
                        self.nbins = int(data)
                    else:
                        self.tbins = data.split()
                        self.tbins = [float(x) for x in self.tbins]
                        self.coefficients['CC_tbins'] = json.dumps(self.tbins)
                
                elif 'C and A offset' in comment:
                    data = data.split()
                    self.cwlngth.append(float(data[0][1:]))
                    self.awlngth.append(float(data[1][1:]))
                    self.ccwo.append(float(data[3]))
                    self.acwo.append(float(data[4]))
                    tcrow = [float(x) for x in data[5:self.nbins+5]]
                    tarow = [float(x) for x in data[self.nbins+5:2*self.nbins+5]]
                    self.tcarray.append(tcrow)
                    self.taarray.append(tarow)
                    self.coefficients['CC_cwlngth'] = json.dumps(self.cwlngth)
                    self.coefficients['CC_awlngth'] = json.dumps(self.awlngth)
                    self.coefficients['CC_ccwo'] = json.dumps(self.ccwo)
                    self.coefficients['CC_acwo'] = json.dumps(self.acwo)
            
                else:
                    pass
            
            else:
                pass
            
            
    def write_csv(self, savepath):
        """
        This function writes the correctly named csv file for the ctd to the
        specified directory.
    
        Args:
            outpath - directory path of where to write the csv file
        Raises:
            ValueError - raised if the OPTAA's object's coefficient dictionary 
                has not been populated
        Returns:
            self.to_csv - a csv of the calibration coefficients which is 
                written to the specified directory from the outpath.
        """
        # Now, write to a csv file
        # Create a dataframe to write to the csv
        data = {
            'serial':self.serial,
            'name':list(self.coefficients.keys()),
            'value':list(self.coefficients.values()),
            'notes':['']*len(self.coefficients)
        }
        
        df = pd.DataFrame().from_dict(data)
        
        # Generate the cal csv filename
        filename = self.uid + '__' + self.date + '.csv'
        # Now write to 
        check = input(f"Write {filename} to {savepath}? [y/n]: ")
        if check.lower().strip() == 'y':
            df.to_csv(savepath+'/'+filename, index=False)
        
        # Generate the tc and ta array filename
        tc_name = filename + '__CC_tcarray.ext'
        ta_name = filename + '__CC_taarray.ext'
        
        def write_array(filename, array):
            with open(filename, 'w') as out:
                array_writer = csv.writer(out)
                array_writer.writerows(array)
                
        write_array(savepath+'/'+tc_name, self.tcarray)
        write_array(savepath+'/'+ta_name, self.taarray)

In [3]:
qct_directory = '/media/andrew/OS/Users/areed/Documents/Project_Files/'
cal_directory = '/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/OPTAA/'
asset_management_directory = '/home/andrew/Documents/OOI-CGSN/ooi-integration/asset-management/calibration/OPTAA/'

In [4]:
excel_spreadsheet = '/media/andrew/OS/Users/areed/Documents/Project_Files/Documentation/System/System Notebook/WHOI_Asset_Tracking.xlsx'
sheet_name = 'Sensors'

In [5]:
OPTAA = whoi_asset_tracking(spreadsheet=excel_spreadsheet,sheet_name=sheet_name,instrument_class='OPTAA')

In [24]:
uid,sn = serial_nums.items()

ValueError: too many values to unpack (expected 2)

In [ ]:
OPTAA

In [31]:
uids = list(set(OPTAA['UID']))
qct_dict = {}
for uid in uids:
    # Get the QCT Document numbers from the asset tracking sheet
    OPTAA['UID_match'] = OPTAA['UID'].apply(lambda x: True if uid in x else False)
    qct_series = OPTAA[OPTAA['UID_match'] == True]['QCT Testing']
    qct_series = list(qct_series.iloc[0].split('\n'))
    qct_dict.update({uid:qct_series})

In [33]:
uids

['CGINS-OPTAAD-00129',
 'CGINS-OPTAAD-00123',
 'CGINS-OPTAAD-00153',
 'CGINS-OPTAAD-00255',
 'CGINS-OPTAAD-00165',
 'CGINS-OPTAAD-00205',
 'CGINS-OPTAAD-00241',
 'CGINS-OPTAAD-00150',
 'CGINS-OPTAAD-00206',
 'CGINS-OPTAAD-00187',
 'CGINS-OPTAAD-00222',
 'CGINS-OPTAAD-00245',
 'CGINS-OPTAAD-00256',
 'CGINS-OPTAAD-00257',
 'CGINS-OPTAAD-00218',
 'CGINS-OPTAAD-00240',
 'CGINS-OPTAAD-00152',
 'CGINS-OPTAAD-00209',
 'CGINS-OPTAAD-00254',
 'CGINS-OPTAAD-00189',
 'CGINS-OPTAAD-00207',
 'CGINS-OPTAAD-00193',
 'CGINS-OPTAAD-00130',
 'CGINS-OPTAAD-00242',
 'CGINS-OPTAAD-00185',
 'CGINS-OPTAAD-00159',
 'CGINS-OPTAAD-00151']

In [34]:
serial_nums = get_serial_nums(OPTAA, uids)

In [35]:
for key in serial_nums.keys():
    values = [str(x) for x in serial_nums[key]]
    serial_nums[key] = values
    print(values)

['129']
['123']
['153']
['255']
['165']
['205']
['241']
['150']
['206']
['187']
['222']
['245']
['256']
['257']
['218']
['240']
['152']
['209']
['254']
['189']
['207']
['193']
['130']
['242']
['185']
['159']
['151']


In [36]:
serial_nums

{'CGINS-OPTAAD-00129': ['129'],
 'CGINS-OPTAAD-00123': ['123'],
 'CGINS-OPTAAD-00153': ['153'],
 'CGINS-OPTAAD-00255': ['255'],
 'CGINS-OPTAAD-00165': ['165'],
 'CGINS-OPTAAD-00205': ['205'],
 'CGINS-OPTAAD-00241': ['241'],
 'CGINS-OPTAAD-00150': ['150'],
 'CGINS-OPTAAD-00206': ['206'],
 'CGINS-OPTAAD-00187': ['187'],
 'CGINS-OPTAAD-00222': ['222'],
 'CGINS-OPTAAD-00245': ['245'],
 'CGINS-OPTAAD-00256': ['256'],
 'CGINS-OPTAAD-00257': ['257'],
 'CGINS-OPTAAD-00218': ['218'],
 'CGINS-OPTAAD-00240': ['240'],
 'CGINS-OPTAAD-00152': ['152'],
 'CGINS-OPTAAD-00209': ['209'],
 'CGINS-OPTAAD-00254': ['254'],
 'CGINS-OPTAAD-00189': ['189'],
 'CGINS-OPTAAD-00207': ['207'],
 'CGINS-OPTAAD-00193': ['193'],
 'CGINS-OPTAAD-00130': ['130'],
 'CGINS-OPTAAD-00242': ['242'],
 'CGINS-OPTAAD-00185': ['185'],
 'CGINS-OPTAAD-00159': ['159'],
 'CGINS-OPTAAD-00151': ['151']}

In [39]:
cal_dict = get_calibration_files(serial_nums, cal_directory)
cal_dict

129
123
153
255
165
205
241
150
206
187
222
245
256
257
218
240
152
209
254
189
207
193
130
242
185
159
151


{'CGINS-OPTAAD-00129': ['OPTAA-D_AC-S_SN_129_Calibration_Files_2015-10-05.zip',
  'OPTAA-D_AC-S_SN_129_Calibration_Files_2017-02-09.zip',
  'OPTAA-D_AC-S_SN_129_Calibration_Files_2018-04-11.zip',
  'OPTAA-D_AC-S_SN_129_Calibration_Files_2013-01-30.zip'],
 'CGINS-OPTAAD-00123': ['OPTAA-D_AC-S_SN_123_Calibration_Files.zip',
  'OPTAA-D_AC-S_SN_123_Calibration_Files_2013-07-17.zip',
  'OPTAA-D_AC-S_SN_123_Calibration_Files_2015-07-20.zip',
  'OPTAA-D_AC-S_SN_123_Calibration_Files_2016-09-29.zip'],
 'CGINS-OPTAAD-00153': ['OPTAA-D_AC-S_SN_153_Calibration_Files_2013-12-12.zip',
  'OPTAA-D_AC-S_SN_153_Calibration_Files_2016-08-16.zip'],
 'CGINS-OPTAAD-00255': ['OPTAA-D_AC-S_SN_255_Calibration_Files_2015-08-04.zip',
  'OPTAA-D_AC-S_SN_255_Calibration_Files_2017-07-11.zip',
  'OPTAA-D_AC-S_SN_255_Calibration_Files_2018-11-15.zip'],
 'CGINS-OPTAAD-00165': ['OPTAA-D_AC-S_SN_165_Calibration_Files.zip',
  'OPTAA-D_AC-S_SN_165_Calibration_Files_2016-01-07.zip',
  'OPTAA-D_AC-S_SN_165_Calibration_Fil

In [42]:
# Lets try opening/loading
uid = uids[0]
uid

'CGINS-OPTAAD-00129'

In [43]:
cal_dict[uid]

['OPTAA-D_AC-S_SN_129_Calibration_Files_2015-10-05.zip',
 'OPTAA-D_AC-S_SN_129_Calibration_Files_2017-02-09.zip',
 'OPTAA-D_AC-S_SN_129_Calibration_Files_2018-04-11.zip',
 'OPTAA-D_AC-S_SN_129_Calibration_Files_2013-01-30.zip']

In [44]:
filepath = generate_file_path(cal_directory, 'OPTAA-D_AC-S_SN_129_Calibration_Files_2015-10-05', ext=['.zip'])

In [45]:
filepath

'/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/OPTAA/OPTAA-D_AC-S_SN_129_Calibration_Files_2015-10-05.zip'

In [94]:
import string

In [262]:
optaa = OPTAACalibration(uid=uid)

In [263]:
optaa.load_dev(filepath)

In [264]:
optaa.write_csv(optaa_savepath)

Write CGINS-OPTAAD-00129__20150930.csv to /home/andrew/Documents/OOI-CGSN/QAQC_Sandbox/Metadata_Review/temp/optaa? [y/n]: y


In [249]:
import shutil

In [253]:
# Purge the temp directory
try:
    shutil.rmtree('/'.join((os.getcwd(),'temp')))
except:
    pass
# Put the csv files into a similar temp directory for local working
ensure_dir('/'.join((os.getcwd(),'temp','optaa')))
optaa_savepath = '/'.join((os.getcwd(),'temp','optaa'))

In [257]:
optaa_savepath

'/home/andrew/Documents/OOI-CGSN/QAQC_Sandbox/Metadata_Review/temp/optaa'

In [246]:
optaa.coefficients

{'CC_taarray': 'SheetRef:CC_taarray',
 'CC_tcarray': 'SheetRef:CC_tcarray',
 'CC_tbins': '[3.912083, 4.359483, 5.456582, 6.47945, 7.488736, 8.494507, 9.485833, 10.491887, 11.492, 12.483182, 13.479512, 14.478, 15.492105, 16.496857, 17.482424, 18.477273, 19.496364, 20.505333, 21.500345, 22.488519, 23.487778, 24.498077, 25.4912, 26.500769, 27.5168, 28.5004, 29.487692, 30.496786, 31.502069, 32.490909, 33.487714, 34.493158]',
 'CC_cwlngth': '[399.6, 404.3, 408.5, 412.5, 417.1, 421.7, 426.9, 431.5, 435.6, 440.0, 444.9, 450.4, 455.0, 459.6, 464.3, 469.2, 474.5, 479.7, 484.8, 489.5, 494.1, 498.8, 503.4, 508.8, 513.9, 519.2, 524.2, 529.1, 533.7, 538.4, 543.2, 548.0, 552.9, 557.7, 562.8, 567.4, 571.9, 576.1, 580.2, 584.4, 587.8, 592.3, 596.7, 601.1, 605.8, 610.5, 615.2, 619.5, 624.0, 628.4, 632.9, 637.0, 641.5, 645.8, 650.4, 654.9, 659.5, 663.9, 668.2, 672.4, 676.5, 680.7, 684.5, 688.7, 692.3, 696.0, 699.4, 703.1, 706.5, 710.0, 713.7, 716.9, 720.0, 723.3, 726.7, 729.4, 732.6, 735.1, 738.1, 740.8

In [230]:
if filepath.endswith('.zip'):
    with ZipFile(filepath) as zfile:
        filename = [name for name in zfile.namelist() if name.endswith('.dev')]
        text = zfile.read(filename[0]).decode('ASCII')

# Remove extraneous characters from the 
punctuation = ''.join((letter for letter in string.punctuation if letter not in ';/.'))
        
for line in text.replace('\t',' ').splitlines():
    line = ''.join((word for word in line if word not in punctuation))
            
    if 'tcal' in line:
        data = line.split()
        # Temperature calibration value
        tcal = data.index('tcal')
        optaa.tcal = data[tcal+1]
        # Temperature calibration date
        cal_date = data[-1].strip()
        optaa.date = pd.to_datetime(cal_date).strftime('%Y%m%d')
            
    elif ';' in line:
        data, comment = line.split(';')
        
        if 'temperature bins' in comment:
            if 'number' in comment:
                optaa.nbins = int(data)
            else:
                optaa.tbins = data.split()
                optaa.tbins = [float(x) for x in optaa.tbins]
                optaa.coefficients['CC_tbins'] = json.dumps(optaa.tbins)
                
        elif 'C and A offset' in comment:
            data = data.split()
            optaa.cwlngth.append(float(data[0][1:]))
            optaa.awlngth.append(float(data[1][1:]))
            optaa.ccwo.append(float(data[3]))
            optaa.acwo.append(float(data[4]))
            tcrow = [float(x) for x in data[5:optaa.nbins+5]]
            tarow = [float(x) for x in data[optaa.nbins+5:2*optaa.nbins+5]]
            optaa.tcarray.append(tcrow)
            optaa.taarray.append(tarow)
            optaa.coefficients['CC_cwlngth'] = json.dumps(optaa.cwlngth)
            optaa.coefficients['CC_awlngth'] = json.dumps(optaa.awlngth)
            optaa.coefficients['CC_ccwo'] = json.dumps(optaa.ccwo)
            optaa.coefficients['CC_acwo'] = json.dumps(optaa.acwo)
    
        else:
            pass
            
    else:
        pass
            
            


TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'

In [241]:
for line in text.replace('\t',' ').splitlines():
    
    if ';' in line:
        print(line)
        data, comment = line.split(';')
    
        if 'temperature bins' in comment:
            if 'number' in comment:
                print(data + '::' + comment)
                nbins = int(data)
            else:
                tbins = data.split()
                tbins = [float(x) for x in optaa.tbins]
            

53000081  ; Serial number                                                                      
3 ; structure version number                                                                       
0 0  ; Depth calibration                                                                     
115200   ; Baud rate                                                                     
0.25   ; Path length (meters)                                                                     
83   ; output wavelengths                                                                     
32   ; number of temperature bins                                                                     
32   :: number of temperature bins                                                                     
     3.912083 4.359483 5.456582 6.47945 7.488736 8.494507 9.485833 10.491887 11.492 12.483182 13.479512 14.478 15.492105 16.496857 17.482424 18.477273 19.496364 20.505333 21.500345 22.488519 23.487778 24.498077 25.4912 

In [242]:
nbins

32

In [197]:
optaa.date

In [194]:
optaa.coefficients.values()

dict_values(['SheetRef:CC_taarray', 'SheetRef:CC_tcarray'])

In [ ]:
with open(filename) as file